# Config stuff

In [1]:
from numpy.ma.core import inner

import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("FACT_RIDES")
spark.getActiveSession()

# Make tables available

In [3]:
#EXTRACT
# Dimension date
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")

# Dimension weather
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")

# Fact rides
fact_rides = spark.read.format("delta").load("spark-warehouse/factrides")

# Make tempviews from tables
dim_date.createOrReplaceTempView("dimDate")
dim_weather.createOrReplaceTempView("dimWeather")
fact_rides.createOrReplaceTempView("factRides")

# Analysis queries

## Student 1

### Wat zijn de drukke momenten (op dagbasis) in de week t.o.v. het weekend?

In [18]:
spark.sql("select dd.DayOfWeekStartMonday, count(fr.count_MV) as CountedRides from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by dd.DayOfWeekStartMonday \
          order by dd.DayOfWeekStartMonday asc ").show()

+--------------------+------------+
|DayOfWeekStartMonday|CountedRides|
+--------------------+------------+
|                   1|      677374|
|                   2|      717198|
|                   3|      513832|
|                   4|      403862|
|                   5|      398042|
|                   6|      511975|
|                   7|      701441|
+--------------------+------------+



In de week zijn vooral maandag en dinsdag de drukste dagen. Daarna neemt het aantal rides af elke dag.
In het weekend neemt het aantal rides terug toe en is het op zijn drukst op zondag.

### Hebben datumparameters invloed op de afgelegde afstand?

### Heeft weer invloed op ritten?

Op welke eigenschappen van de ritten kan het weer een invloed hebben:
1) Op aantal afgelegde ritten in een bepaald weertype
2) Op de gemiddelde duurtijd van een rit in een bepaald weertype
3) Op de gemiddelde afgelegde afstand van een rit in een bepaald weertype

In [30]:
spark.sql("select dw.weather_type, count(count_MV) as CountedRides, \
          round(sum(rideDuration_MV)/count(count_MV)) as AverageRideDuration \
          from factRides fr \
          inner join dimWeather dw on fr.weather_SK = dw.weather_SK \
          group by dw.weather_type \
          having dw.weather_type != 'weertype onbekend'").show()

+------------+------------+-------------------+
|weather_type|CountedRides|AverageRideDuration|
+------------+------------+-------------------+
|    Neutraal|         208|              677.0|
|   Aangenaam|         197|              718.0|
| Onaangenaam|          90|              635.0|
+------------+------------+-------------------+



#### 1. Invloed van weer op aantal afgelegde ritten
Op basis van de analyse hierboven zouden we kunnen zeggen dat bij neutraal of aangenaam weer er de meeste ritten worden afgewerkt.

#### 2. Invloed van weer op de duurtijd van een rit
We kunnen uit de analyse afleiden dat gebruikers bij aangenaam weer gemiddeld gezien de langste ritten afleggen op vlak van duurtijd. Bij onaangenaam weer zitten gebruikers het minst lang op hun voertuig.

In praktijk weten we echter dat we random weer situaties hebben toegevoegd aan onze data. Dus de patronenen die we hier zien zijn louter toeval.

#### 3. Invloed van weer op de afgelegde afstand

### Stijgt de populariteit van deelfietsen in Antwerpen met de jaren?

In [19]:
spark.sql("select dd.CalendarYear as Year, count(fr.count_MV) as CountedRides from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by dd.CalendarYear").show()

+----+------------+
|Year|CountedRides|
+----+------------+
|2019|      288379|
|2021|     1013666|
|2020|      958186|
|2022|      958269|
|2023|      705224|
+----+------------+



### Hebben de seizoenen invloed op de ritten?

1. Hoeveel ritten zijn er per seizoen?
2. Wat is de gemiddelde duurtijd per rit per seizoen?
3. Wat is de gemiddelde afgelegde afstand per rit per seizoen

In [34]:
spark.sql("select CASE \
          when (dd.MonthOfYear = 12 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (1,2)) or (dd.MonthOfYear = 3 and dd.DayOfMonth <21) then 'winter' \
          when (dd.MonthOfYear = 3 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (4,5)) or (dd.MonthOfYear = 6 and dd.DayOfMonth <21) then 'spring' \
          when (dd.MonthOfYear = 6 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (7,8)) or (dd.MonthOfYear = 9 and dd.DayOfMonth <21) then 'summer' \
          when (dd.MonthOfYear = 9 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (10,11)) or (dd.MonthOfYear = 12 and dd.DayOfMonth <21) then 'autumn' \
          end as season, \
          count(fr.count_MV) as CountedRides, \
          round(sum(fr.rideDuration_MV)/count(fr.count_MV)) as AverageRideDuration \
          from factRides fr \
          inner join dimDate dd on fr.date_SK = dd.date_SK \
          group by \
          CASE \
          when (dd.MonthOfYear = 12 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (1,2)) or (dd.MonthOfYear = 3 and dd.DayOfMonth <21) then 'winter' \
          when (dd.MonthOfYear = 3 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (4,5)) or (dd.MonthOfYear = 6 and dd.DayOfMonth <21) then 'spring' \
          when (dd.MonthOfYear = 6 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (7,8)) or (dd.MonthOfYear = 9 and dd.DayOfMonth <21) then 'summer' \
          when (dd.MonthOfYear = 9 and dd.DayOfMonth >=21) or (dd.MonthOfYear in (10,11)) or (dd.MonthOfYear = 12 and dd.DayOfMonth <21) then 'autumn' \
          end \
          order by CountedRides desc").show()

+------+------------+-------------------+
|season|CountedRides|AverageRideDuration|
+------+------------+-------------------+
|autumn|      993222|              609.0|
|summer|      992891|              603.0|
|spring|      988528|              603.0|
|winter|      949083|              603.0|
+------+------------+-------------------+



1. We zien dat er in de herfst de meeste ritten zijn. Al is het verschil met de andere seizoenen beperkt.
2. de gemiddelde duurtijd per rit is in de herfst het hoogst. Bij de andere seizoenen is deze gelijk.

## Student 2

### Wat is de invloed van de woonplaats van de gebruikers op het gebruik van de vehicles?

### We willen voorspellen welke sloten preventief onderhoud nodig hebben. Bekijk hoe vaak slotnummers relatief gezien gebruikt worden.

### Als een klant zijn abonnement stopzet, willen we kunnen voorspellen op welke stations dit het meeste effect zal hebben.

### Extra vraag 1

### Extra vraag 2